In [3]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


import joblib

In [21]:
filename = 'events.csv'
events = pd.read_csv('./data/' + filename)
shots = events[(events.event_type==1)]
shots_on_target = shots[(shots.shot_outcome==1)].copy()
shots_on_target['is_save'] = 1 - shots_on_target['is_goal']
shots_on_target.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,is_save
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0,0
22,UFot0hit/,UFot0hit23,23,26,Attempt saved. Mats Hummels (Borussia Dortmund...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,0,3.0,3.0,2,1.0,0,1
26,UFot0hit/,UFot0hit27,27,29,"Goal! Borussia Dortmund 2, Hamburg 0. Mario G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,1,3.0,1.0,1,1.0,0,0
49,UFot0hit/,UFot0hit50,50,48,Attempt saved. Kevin Grosskreutz (Borussia Dor...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,11.0,1.0,0,3.0,1.0,1,1.0,0,1
50,UFot0hit/,UFot0hit51,51,49,"Goal! Borussia Dortmund 3, Hamburg 0. Kevin G...",1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,13.0,1.0,0,3.0,0,0


In [71]:
shots_prediction = shots_on_target.iloc[:,[-1,-6,-5,-4,-3, -2]]
#shots_prediction.head()
dummies = pd.get_dummies(shots_prediction, columns=['location', 'bodypart','assist_method', 'situation'])
dummies.columns = ['is_save', 'fast_break','loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right', 'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range', 'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 'left_foot', 'header', 'no_assist', 'assist_pass', 'assist_cross', 'assist_header', 'assist_through_ball', 'open_play', 'set_piece', 'corner', 'free_kick']
dummies.head()

,is_save,fast_break,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,...,header,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick
13,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22,1,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
26,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
49,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
50,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [72]:
X = dummies.iloc[:,1:]
y = dummies.iloc[:,0]
print(X.shape)
print(y.shape)

(78014, 28)
(78014,)


In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

In [76]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(28, 28, 28, 28), random_state=1, max_iter=10000)
clf.fit(X, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(28, 28, 28, 28), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [77]:
accuracy = clf.score(X_test, y_test)
y_pred = clf.predict_proba(X_test)
predict = clf.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
auc_roc = roc_auc_score(y_test, y_pred[:, 1])
print('The training set contains {} examples (shots on target) of which {} are positives (goals).'.format(y_total, y_positive))
print('The accuracy of classifying whether a shot is goal or not is {:.2f} %'.format(accuracy*100))
print('Our classifier obtains an AUC-ROC of {:.4f}.'.format(auc_roc))

The training set contains 50709 examples (shots on target) of which 35195 are positives (goals).
The accuracy of classifying whether a shot is goal or not is 81.61 %
Our classifier obtains an AUC-ROC of 0.8378.


In [78]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict))
print('Report:')
print(classification_report(y_test,predict))

Confusion Matrix:
[[ 3740  4562]
 [  460 18543]]
Report:
              precision    recall  f1-score   support

           0       0.89      0.45      0.60      8302
           1       0.80      0.98      0.88     19003

    accuracy                           0.82     27305
   macro avg       0.85      0.71      0.74     27305
weighted avg       0.83      0.82      0.79     27305



In [81]:
predictions = X_test.copy()
predictions['true_saves'] = y_test
predictions['expected_saves'] = y_pred[:,1]
predictions['difference'] = predictions['expected_saves'] - predictions['true_saves']
predictions = predictions.iloc[:,28:31]

In [82]:
ypred2 = clf.predict_proba(X_train)
predictions_train = X_train.copy()
predictions_train['true_saves'] = y_train
predictions_train['expected_saves'] = ypred2[:,1]
predictions_train['difference'] = predictions_train['expected_saves'] - predictions_train['true_saves']
predictions_train = predictions_train.iloc[:,28:31]
all_predictions = pd.concat([predictions, predictions_train], axis=0)
events2 = pd.concat([events, all_predictions], axis=1)
shots2 = events2[events2.event_type==1]

In [87]:
xG_Teams = shots2[['opponent', 'event_type', 'true_saves', 'expected_saves', 'difference']].groupby('opponent').sum()
xG_Teams.columns = ['n_shots', 'saves_made', 'expected_saves', 'difference']
xG_Teams[['saves_made', 'expected_saves']].corr()

,saves_made,expected_saves
saves_made,1.000000,0.997743
expected_saves,0.997743,1.000000


In [89]:
xG_Teams.sort_values(['difference', 'saves_made'])

,n_shots,saves_made,expected_saves,difference
opponent,,,,
Juventus,1997,448.0,416.184882,-31.815118
Bayern Munich,1432,380.0,350.098040,-29.901960
Athletic Bilbao,2494,627.0,597.225461,-29.774539
Genoa,2938,702.0,673.014010,-28.985990
Paris Saint-Germain,2228,503.0,476.682892,-26.317108
...,...,...,...,...
Liverpool,1366,293.0,315.610710,22.610710
Lorient,2485,638.0,661.745347,23.745347
Rayo Vallecano,2332,550.0,573.766854,23.766854


In [91]:
xG_Teams.sort_values(['expected_saves'], ascending=False)

,n_shots,saves_made,expected_saves,difference
opponent,,,,
Genoa,2938,702.0,673.014010,-28.985990
Lorient,2485,638.0,661.745347,23.745347
Hamburg SV,2588,664.0,655.785505,-8.214495
Atalanta,2894,665.0,655.146857,-9.853143
Real Sociedad,2609,643.0,645.436608,2.436608
...,...,...,...,...
Crotone,319,90.0,88.319993,-1.680007
Middlesbrough,304,61.0,58.937737,-2.062263
Alaves,262,57.0,51.128068,-5.871932


In [94]:
xG_Teams['xG_per_shot_ratio'] = xG_Teams['expected_saves'] / xG_Teams['n_shots']
xG_Teams[xG_Teams.n_shots>1].sort_values(['xG_per_shot_ratio', 'saves_made'])

,n_shots,saves_made,expected_saves,difference,xG_per_shot_ratio
opponent,,,,,
Middlesbrough,304,61.0,58.937737,-2.062263,0.193874
Alaves,262,57.0,51.128068,-5.871932,0.195145
Frosinone,739,142.0,146.232485,4.232485,0.197879
Southampton,1364,274.0,282.144612,8.144612,0.206851
Leicester City,1376,287.0,284.799011,-2.200989,0.206976
...,...,...,...,...,...
Hannover 96,2303,617.0,621.976194,4.976194,0.270072
Eintracht Frankfurt,1868,525.0,512.255132,-12.744868,0.274227
GFC Ajaccio,386,97.0,106.266965,9.266965,0.275303
